# Домашнее задание № 3

**Предметная область:** медицинская клиника.

**Дано:**
* файл с результатами анализов в зашифрованном виде - medicine.xlsx (файл располагается в одном каталоге с итоговой программой).\
* таблицы med_an_name, med_name для расшифровки показаний (в рамках задания создайте их в своей базе данных, код ниже). 

**Что необходимо сделать:**
1) Режим попроще:
    * забираете данные с листа `easy`
    * вычисляете пациентов, у которых не в порядке хотябы один анализ
    * сохраняете в xlsx имя, телефон, название анализа и заключение: 'Повышен' или 'Понижен'

2) Режим посложнее:
    * забираете данные с листа `hard`
    * вычисляете пациентов, у которых не в порядке два и более анализов
    * сохраняете в xlsx имя, телефон, название анализа и заключение: 'Повышен', 'Понижен' или 'Положительный'

**Что ожидается на выходе:**\
В ответе приложите файл - скрипт python (название easy.py или hard.py в зависимости от выбранного режима)\
Результатом работы скрипта должен быть xlsx файл с заданной информацией.

#### *Создание таблицы с медицинскими показателями*

In [ ]:
CREATE TABLE med_an_name (
	id varchar(10) NULL,
	name varchar(200) NULL,
	is_simple bpchar(1) NULL,
	min_value numeric(18, 2) NULL,
	max_value numeric(18, 2) NULL
);

INSERT INTO med_an_name (id,name,is_simple,min_value,max_value) VALUES
	 ('IG','Иммуноглобулин общий','N',0.00,100.00),
	 ('ALAT','Аланинаминотрансфераза ','N',0.00,45.00),
	 ('2-A','Лейкоциты (моча)','N',0.00,5.00),
	 ('G124','Гаптоглобин','N',150.00,2000.00),
	 ('890','Глюкоза','N',3.30,5.50),
	 ('S','Щелочная фосфатаза','N',40.00,150.00),
	 ('1-100','Общий белок ','N',64.00,84.00),
	 ('N','Нитрит (моча)','Y',NULL,NULL),
	 ('3-511','Креатинин','N',62.00,115.00),
	 ('Z','Эритроциты (моча)','N',0.00,2.00),
	 ('IRR','Липаза','N',0.00,190.00),
	 ('GLK','Глюкоза (моча)','Y',NULL,NULL),
	 ('1-875','Грибки (моча)','Y',NULL,NULL),
	 ('1-900','Амилаза панкреатическая','N',0.00,50.00),
	 ('BBB','Белок (моча)','Y',NULL,NULL),
	 ('AU','Гематокрит','N',39.00,49.00),
	 ('C','СОЭ','N',2.00,20.00),
	 ('aZz','Билирубин','N',5.00,20.00);

#### *Создание таблицы с информацией о клиентах*

In [ ]:
CREATE TABLE med_name (
	id int4 NULL,
	name varchar(50) NULL,
	phone varchar(20) NULL
);

INSERT INTO med_name (id,name,phone) VALUES
	 (191,'Чемиренко Д.А','+7 (905) 912-28-47'),
	 (140,'Головцов Р.Р','+7 (906) 949-98-17'),
	 (119,'Сакобов А.Т','+7 (952) 996-60-21'),
	 (57,'Тарлов И.Е','+7 (900) 875-67-38'),
	 (195,'Свенюков Б.Ю','+7 (983) 577-27-21'),
	 (53,'Попелицкий Р.Г','+7 (976) 841-79-95'),
	 (96,'Умралиев Р.Э','+7 (941) 954-55-88'),
	 (62,'Бондарев С.Г','+7 (906) 933-95-63'),
	 (199,'Цетнарский Э.И','+7 (908) 245-80-47'),
	 (114,'Даклеев Т.М','+7 (925) 612-93-95'),
	 (73,'Гармокацкий С.Л','+7 (935) 653-32-78'),
	 (80,'Маржецкий А.Я','+7 (949) 912-97-84'),
	 (151,'Поляховский Г.Д','+7 (998) 596-45-47'),
	 (127,'Галендук Л.Г','+7 (935) 164-87-46'),
	 (135,'Номоконов Г.Р','+7 (940) 384-74-90'),
	 (77,'Шелкоплясов Э.В','+7 (979) 643-71-75'),
	 (67,'Эльменькин Д.Р','+7 (982) 643-36-73'),
	 (176,'Ламок А.О','+7 (984) 779-54-57'),
	 (163,'Бочечкаров Э.Д','+7 (943) 102-79-69'),
	 (192,'Голяминских Г.Р','+7 (903) 280-60-58'),
	 (72,'Пахарев Ю.Э','+7 (977) 810-48-75'),
	 (111,'Ведмидев Я.У','+7 (997) 541-22-17'),
	 (118,'Сисемкин В.Р','+7 (960) 542-96-83'),
	 (148,'Штыкулин И.А','+7 (949) 862-47-83'),
	 (162,'Шапаев Л.А','+7 (919) 109-43-56'),
	 (76,'Млицкий С.А','+7 (987) 542-73-38'),
	 (101,'Загатин А.Д','+7 (987) 928-64-89');

#### *Решение*

In [15]:
## Решение
import csv
import sqlite3
con = sqlite3.connect(":memory:")  # change to 'sqlite:///your_filename.db'
cur = con.cursor()
# use your column names here
cur.execute("CREATE TABLE big_sales (date, sales, category, additional);")
# `with` statement available in 2.5+
with open('/Users/antontravkin/Sites/python_rtk/part 2/Lesson 6/hw/big_sales.csv', 'r') as fin:
    # csv.DictReader uses first line in file for column headings by default
    dr = csv.DictReader(fin)  # comma is default delimiter
    to_db = [(i['date'], i['sales'], i['category'], i['additional']) for i in dr]
cur.executemany(
    "INSERT INTO big_sales (date, sales, category, additional) VALUES (?, ?, ?, ?);", to_db)
con.commit()
con.close()

KeyError: 'date'